In [1]:
import numpy as np

import astropy.table as tb 
from astropy.time import Time  
from astropy import units as u

from spacerocks.units import Units
from spacerocks.simulation import Simulation
from spacerocks.model import PerturberModel, builtin_models
from spacerocks.cbindings import correct_for_ltt_destnosim
from spacerocks.observer import Observer
from spacerocks.constants import epsilon

In [5]:
from utils import createRandomObjects
import destnosim

In [6]:
mjd = [52222, 53333]
objs = createRandomObjects(100)
population = destnosim.ElementPopulation(objs, 16)
times = Time(mjd, format='mjd', scale='utc')
rocks = population.generateSpaceRocks()

In [10]:
times.jd

array([2452222.5, 2453333.5])

In [19]:
obs = Observer.from_obscode('W84').at(times.jd[0])
obs.x.au.astype(np.double)

array([0.67823096])

In [ ]:


def c1(population, mjd, progress=False):
	'''
	Calls the Spacerocks backend to generate observations for the input population


	Arguments:
	- population: Population object for the input orbits
	'''
	## first set up times and do spacerock stuff

	#self.createEarthSpaceRock()
	times = Time(mjd, format='mjd', scale='utc')
	rocks = population.generateSpaceRocks()

	units = Units()
	units.timescale = 'utc'
	units.timeformat = 'jd'
	units.mass = u.M_sun
    
	spiceids, kernel, masses = builtin_models['ORBITSPP']
	model = PerturberModel(spiceids=spiceids, masses=masses)
	
	sim = Simulation(model=model, epoch=times.jd[0], units=units)
	sim.add_spacerocks(rocks)
	sim.integrator = 'leapfrog'

	ras = np.array([])
	decs = np.array([])
	orbitid = np.array([])
	mjds = np.array([])
	oidlist = np.arange(len(population))
	xList = np.array([])
	yList = np.array([])
	zList = np.array([])
	dList = np.array([])

	if progress == True:
		from rich.progress import track
		epochs = track(range(len(times.jd)))
	else:
		epochs = range(len(times.jd))

	for i in epochs:
		sim.integrate(times.jd[i], exact_finish_time=1)
		a = np.zeros((sim.N, 3), dtype=np.double)
		b = np.zeros((sim.N, 3), dtype=np.double)
		sim.serialize_particle_data(xyz=a, vxvyvz=b)
		x, y, z = a.T
		vx, vy, vz = b.T
    
		x = np.ascontiguousarray(x)[sim.N_active:]
		y = np.ascontiguousarray(y)[sim.N_active:]
		z = np.ascontiguousarray(z)[sim.N_active:]
		vx = np.ascontiguousarray(vx)[sim.N_active:]
		vy = np.ascontiguousarray(vy)[sim.N_active:]
		vz = np.ascontiguousarray(vz)[sim.N_active:]

		np.append(xList, x)
		np.append(yList, y)
		np.append(zList, z)
		np.append(dList, np.sqrt(x**2 + y**2 + z**2))

		# observer = Observer(epoch=times.jd[i], obscode='W84', units=units)
		observer = Observer.from_obscode('W84').at(times.jd[i])
		ox = observer.x.au.astype(np.double)
		oy = observer.y.au.astype(np.double)
		oz = observer.z.au.astype(np.double)
		ovx = observer.vx.value.astype(np.double)
		ovy = observer.vy.value.astype(np.double)
		ovz = observer.vz.value.astype(np.double)
		
		# Compute ltt-corrected topocentroc Ecliptic coordinates
		xt, yt, zt = correct_for_ltt_destnosim(x, y, z, vx, vy, vz, ox, oy, oz, ovx, ovy, ovz)
		lon = np.arctan2(yt, xt)
		lat = np.arcsin(zt / np.sqrt(xt**2 + yt**2 + zt**2))
		dec = np.degrees(np.arcsin(np.sin(lat) * np.cos(epsilon) + np.cos(lat) * np.sin(lon) * np.sin(epsilon)))
		ra = np.degrees(np.arctan2((np.cos(lat) * np.cos(epsilon) * np.sin(lon) - np.sin(lat) * np.sin(epsilon)), np.cos(lon) * np.cos(lat)))

		ra[ra>180] -= 360

		orbitid = np.append(orbitid, oidlist)
		mjds = np.append(mjds, len(oidlist) * [i])

		ras = np.append(ras, ra)
		decs = np.append(decs, dec)


	del x, y, z, vx, vy, vz, a, b, sim, xt, yt, zt, ox, oy, oz, observer
	## gather data into something useable
	t = tb.Table()
	t['AstRA(deg)'] = ras
	del ras
	#t['RA'][t['RA'] > 180] -= 360
	t['AstDec(deg)'] = decs
	del decs
	t['ObjID'] = orbitid.astype('int64')
	del orbitid

	t['FieldMJD'] = mjds
	t['Mag'] = 20

	t['d'] = dList
	t['x'] = xList
	t['y'] = yList
	t['z'] = zList

	# ObjID,FieldID,FieldMJD,AstRange(km),AstRangeRate(km/s),AstRA(deg),AstRARate(deg/day),AstDec(deg),AstDecRate(deg/day),Ast-Sun(J2000x)(km),Ast-Sun(J2000y)(km),Ast-Sun(J2000z)(km),Ast-Sun(J2000vx)(km/s),Ast-Sun(J2000vy)(km/s),Ast-Sun(J2000vz)(km/s),Obs-Sun(J2000x)(km),Obs-Sun(J2000y)(km),Obs-Sun(J2000z)(km),Obs-Sun(J2000vx)(km/s),Obs-Sun(J2000vy)(km/s),Obs-Sun(J2000vz)(km/s),Sun-Ast-Obs(deg),V,V(H=0),fiveSigmaDepth,filter,MaginFilterTrue,AstrometricSigma(mas),PhotometricSigma(mag),SNR,AstrometricSigma(deg),MaginFilter,dmagDetect,dmagVignet,AstRATrue(deg),AstDecTrue(deg),detector,OBSCODE,NA

	return t

t = c1(population, mjd, progress=True)